# **Olympic Swimming History (1912 to 2020) - Exploratory Data Analysis**

Dataset - [Olympic Swimming History (1912 to 2020)](https://www.kaggle.com/datasets/datasciencedonut/olympic-swimming-1912-to-2020)

In [1]:
# imports

import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

In [2]:
# load dataset

df = pd.read_csv('/kaggle/input/olympic-swimming-1912-to-2020/Olympic_Swimming_Results_1912to2020.csv')

In [3]:
# first 5 records of the dataset

df.head()

,Location,Year,Distance (in meters),Stroke,Relay?,Gender,Team,Athlete,Results,Rank
0,Tokyo,2020,100m,Backstroke,0,Men,ROC,Evgeny Rylov,51.98,1
1,Tokyo,2020,100m,Backstroke,0,Men,ROC,Kliment Kolesnikov,52,2
2,Tokyo,2020,100m,Backstroke,0,Men,USA,Ryan Murphy,52.19,3
3,Tokyo,2020,100m,Backstroke,0,Men,ITA,Thomas Ceccon,52.3,4
4,Tokyo,2020,100m,Backstroke,0,Men,CHN,Jiayu Xu,52.51,4


In [4]:
# to get number of rows and columns along with column names and its types

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4359 entries, 0 to 4358
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Location              4359 non-null   object
 1   Year                  4359 non-null   int64 
 2   Distance (in meters)  4359 non-null   object
 3   Stroke                4359 non-null   object
 4   Relay?                4359 non-null   int64 
 5   Gender                4359 non-null   object
 6   Team                  4359 non-null   object
 7   Athlete               4345 non-null   object
 8   Results               4331 non-null   object
 9   Rank                  4359 non-null   int64 
dtypes: int64(3), object(7)
memory usage: 340.7+ KB


In [5]:
# rename column

df = df.rename(columns={'Distance (in meters)' : 'Distance'})

# change type

df = df.astype({'Year': str, 'Rank' : str})

In [6]:
# check for null values

df.isna().sum()

Location     0
Year         0
Distance     0
Stroke       0
Relay?       0
Gender       0
Team         0
Athlete     14
Results     28
Rank         0
dtype: int64

In [7]:
# remove null values from main dataframe

na = df[df.isna().any(axis=1)]
df.dropna(inplace=True)

# drop rank 0 and rank 4 from the df

rank0 = df[df['Rank'] == '0']
df.drop(rank0.index, inplace=True)

rank4 = df[df['Rank'] == '4']
df.drop(rank4.index, inplace=True)

In [8]:
# grouping Stroke and Distance

grouped = df.groupby(['Stroke', 'Distance']).size().reset_index(name='Count')

In [9]:
# stacked bar chart for better understanding of stroke and distance categories count

fig = px.bar(grouped, x='Distance', y='Count', color='Stroke',
             text='Count', title='Category records')

iplot(fig)

In [10]:
# Relay and Individual

relay = df[df['Relay?'] == 1]
single = df[df['Relay?'] == 0]

In [11]:
# relay events

relay_grouped = relay.groupby(['Team', 'Distance', 'Stroke', 'Rank']).size().reset_index(name='Count')

top_relay = relay_grouped.groupby('Team')['Count'].sum().reset_index().sort_values('Count', ascending=False).head()

country_relay = relay_grouped[relay_grouped['Team'].isin(top_relay['Team'])]

fig = px.bar(country_relay, x='Count', y='Team', color='Rank', text='Count', facet_col='Distance', facet_row='Stroke')

fig.update_layout(yaxis={'categoryorder':'total ascending'}, title='Relay events')

iplot(fig)

In [12]:
# Top 5 countries with the most number of medals in relay

top_relay

,Team,Count
16,USA,86
0,AUS,41
8,GER,39
12,NLD,16
7,GBR,13


In [13]:
# Total number of relay medals by each country

relay_medals = relay.groupby(['Team', 'Rank']).size().reset_index(name='Count')

fig = px.bar(relay_medals, x='Count', y='Team', color='Rank', text='Count', 
             title='Total Relay Medals by country', orientation='h')

fig.update_layout(height=600, yaxis={'categoryorder':'total ascending'})

iplot(fig)

In [14]:
# individual events

single_grouped = single.groupby(['Team', 'Distance', 'Stroke', 'Rank']).size().reset_index(name='Count')

top_single = single_grouped.groupby('Team')['Count'].sum().reset_index().sort_values('Count', ascending=False).head()

country_single = single_grouped[single_grouped['Team'].isin(top_single['Team'])]

fig = px.bar(country_single, x='Count', y='Team', color='Rank', text='Count', facet_row='Distance', facet_col='Stroke')

fig.update_layout(height=800, yaxis={'categoryorder':'total ascending'}, title='Individual events')

iplot(fig)

In [15]:
# Top 5 countries with the most number of medals in individual

top_single

,Team,Count
52,USA,460
2,AUS,156
21,GDR,75
27,JPN,72
24,HUN,59


In [16]:
# Total number of individual medals by each country

single_medals = single.groupby(['Team', 'Rank']).size().reset_index(name='Count')

fig = px.bar(single_medals, x='Count', y='Team', color='Rank', text='Count', 
             title='Total Individual Medals by country')

fig.update_layout(height=1600, yaxis={'categoryorder':'total ascending'})

iplot(fig)

In [17]:
# Top finalist (Country)

pd.concat([df, rank4]).groupby('Team').size().reset_index(name='Count').nlargest(5, 'Count')

,Team,Count
78,USA,881
3,AUS,443
28,GBR,279
40,JPN,242
30,GER,219


In [18]:
# top finalists (athlete)

pd.concat([df, rank4]).groupby(['Athlete', 'Team']).size().reset_index(name='Count').nlargest(5, 'Count')

,Athlete,Team,Count
1796,Michael Phelps,USA,18
1430,Kirsty Leigh Coventry,ZIM,10
1479,Laszlo Cseh,HUN,10
1379,Katie Ledecky,USA,9
2229,Ryan Lochte,USA,9


In [19]:
# athletes with 6 or more medals

athlete = df.groupby(['Athlete', 'Gender','Team']).size().reset_index(name='Count')

athlete[athlete['Count'] > 5].sort_values('Count', ascending=False)

,Athlete,Gender,Team,Count
740,Michael Phelps,Men,USA,16
570,Katie Ledecky,Women,USA,8
604,Krisztina Egerszegi,Women,HUN,7
908,Ryan Lochte,Men,USA,7
446,Inge De Bruijn,Women,NED,6
592,Kirsty Leigh Coventry,Women,ZIM,6
615,Laszlo Cseh,Men,HUN,6
692,Mark Spitz,Men,USA,6


In [20]:
# athletes with 3 or more medals in a category

athletes = df.groupby(['Athlete', 'Gender', 'Team', 'Distance', 'Stroke']).size().reset_index(name='Count')

athletics = athletes[athletes['Count'] >= 3].sort_values(['Distance', 'Stroke'])

fig = px.sunburst(athletics, path=['Distance', 'Stroke', 'Athlete'], values='Count', hover_data=['Team', 'Gender'])

fig.update_layout(title='Athletes with 3 or more medals in a category', height=600)

iplot(fig)

athletics

,Athlete,Gender,Team,Distance,Stroke,Count
955,Michael Phelps,Men,USA,100m,Butterfly,4
338,Duke Paoa Kahanamoku,Men,USA,100m,Freestyle,3
484,Grant Hackett,Men,AUS,1500m,Freestyle,3
750,Kieren John Perkins,Men,AUS,1500m,Freestyle,3
3,Aaron Peirsol,Men,USA,200m,Backstroke,3
779,Krisztina Egerszegi,Women,HUN,200m,Backstroke,3
55,Amanda Beard,Women,USA,200m,Breaststroke,3
511,Halyna Prozumenshchykova-Stepanova,Women,URS,200m,Breaststroke,3
956,Michael Phelps,Men,USA,200m,Butterfly,4
1092,Petria Thomas,Women,AUS,200m,Butterfly,3


In [21]:
# top country year-wise

year = df.groupby(['Year', 'Team']).size().reset_index(name='Count')

year[(year['Count'] >= 20)].sort_values('Year')

,Year,Team,Count
100,1964,USA,29
112,1968,USA,54
127,1972,USA,41
137,1976,USA,34
143,1980,GDR,27
163,1984,USA,33
176,1988,GDR,22
190,1988,USA,20
208,1992,USA,25
229,1996,USA,23


Almost every year after 1964, USA got more than 20 medals and finished first. 

But in 1988, GDR(East Germany) got 22 medals, two more than USA. 

USA boycotted olympics in 1980. 

AUS got 20 medals in 2008.

In [22]:
medals = df.groupby(['Team', 'Rank']).size().reset_index(name='Count').sort_values('Count', ascending=False)
rank1 = medals[medals['Rank'] == '1'].head(10)
rank2 = medals[medals['Rank'] == '2'].head(10)
rank3 = medals[medals['Rank'] == '3'].head(10)

all_medals = df.groupby(['Team']).size().reset_index(name='Count').sort_values('Count', ascending=False).head(10)

In [23]:
fig = make_subplots(rows=2, cols=2, subplot_titles=('Total Medals', 'Gold Medals', 'Silver Medals', 'Bronze Medals'), specs=[[{'type': 'pie'}, {'type': 'pie'}], [{'type': 'pie'}, {'type': 'pie'}]])

list = [0] * 10
list[7] = .3
fig.add_trace(go.Pie(labels=all_medals['Team'], values=all_medals['Count'], textinfo='label+value', hole=0.4, pull=list), 1, 1)

list = [0] * 10
list[6] = .3
fig.add_trace(go.Pie(labels=rank1['Team'], values=rank1['Count'], textinfo='label+value', hole=0.4, pull=list), 1, 2)

list = [0] * 10
list[7] = .3
fig.add_trace(go.Pie(labels=rank2['Team'], values=rank2['Count'], textinfo='label+value', hole=0.4, pull=list), 2, 1)

list = [0] * 10
list[5] = .3
list[9] = .3
fig.add_trace(go.Pie(labels=rank3['Team'], values=rank3['Count'], textinfo='label+value', hole=0.4, pull=list), 2, 2)

fig.update_layout(title='Total medals by country and Number of Gold, Silver & Bronze medals', height=800)

iplot(fig)

# Summary

* In relay events, the top 5 countries are the USA, Australia, Germany, Netherlands and Britain.
 
* For individual events, the top 5 countries are the USA, Australia, East Germany, Japan and Hungary.
 
* The countries with the most number of finalists are the USA, Australia, Britain, Japan and Germany.
 
* The most successful swimmer of all time, Michael Phelps, won a total of 16 medals(13 Gold, 2 Silver and 1 Bronze) in individual events.
 
* USA holds the highest total of 546 medals(Gold-237, Silver-171, Bronze-138), followed by Australia with a total of 197(61, 66 & 70 each)
 
* Canada ranks in the top 10 total medals by country but not in the top 10 gold and silver medals; however, it ranks 6th in the top 10 for Bronze medals.
 
* China has achieved 14 gold medals and 19 silver medals.

* Sweden grips the 10th position in Bronze medals by country.